# Data cleaning - new data obtained at 20190728

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
# %qtconsole

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import display
def display_all(df):
    with pd.option_context("display.max_rows", 1000): 
        with pd.option_context("display.max_columns", 1000): 
            display(df)
			
import pandas as pd
import numpy as np
from pathlib import Path		

import warnings
warnings.filterwarnings('ignore')
# warnings.filterwarnings(action='once')

In [2]:
# ref: https://gis.stackexchange.com/questions/225065/converting-nztm-new-zealand-transverse-mercator-to-lat-long
import math
def nztm_to_lat_long(nztm_e, nztm_n):
    """ converting nztm to latitude and longitude
        #input Northing(Y); Easting(X) variables
    """
    #Common variables for NZTM2000
    a = 6378137;
    f = 1 / 298.257222101;
    phizero = 0;
    lambdazero = 173;
    Nzero = 10000000;
    Ezero = 1600000;
    kzero = 0.9996;   

    #input Northing(Y); Easting(X) variables
    N  = int(nztm_n);
    E  = int(nztm_e);

    #Calculation: From NZTM to lat/Long
    b = a * (1 - f);
    esq = 2 * f - f ** 2;
    Z0 = 1 - esq / 4 - 3 * (esq ** 2) / 64 - 5 * (esq ** 3) / 256;
    A2 = 0.375 * (esq + esq ** 2 / 4 + 15 * (esq ** 3) / 128);
    A4 = 15 * ((esq ** 2) + 3 * (esq ** 3) / 4) / 256;
    A6 = 35 * (esq ** 3) / 3072;

    Nprime = N - Nzero;
    mprime = Nprime / kzero;
    smn = (a - b) / (a + b);
    G = a * (1 - smn) * (1 - (smn ** 2)) * (1 + 9 * (smn ** 2) / 4 + 225 * (smn ** 4) / 64) * math.pi/ 180.0;
    sigma = mprime * math.pi / (180 * G);
    phiprime = sigma + (3 * smn / 2 - 27 * (smn ** 3) / 32) * math.sin(2 * sigma) + (21 * (smn ** 2) / 16 - 55 * (smn ** 4) / 32) * math.sin(4 * sigma) + (151 * (smn ** 3) / 96) * math.sin(6 * sigma) + (1097 * (smn ** 4) / 512) *math.sin(8 * sigma);
    rhoprime = a * (1 - esq) / ((1 - esq * ((math.sin(phiprime)) ** 2)) ** 1.5);
    upsilonprime = a / math.sqrt(1 - esq * ((math.sin(phiprime)) ** 2));

    psiprime = upsilonprime / rhoprime;
    tprime = math.tan(phiprime);
    Eprime = E - Ezero;
    chi = Eprime / (kzero * upsilonprime);
    term_1 = tprime * Eprime * chi / (kzero * rhoprime * 2);
    term_2 = term_1 * (chi ** 2) / 12 * (-4 * (psiprime ** 2) + 9 * psiprime * (1 - (tprime ** 2)) + 12 * (tprime ** 2));
    term_3 = tprime * Eprime * (chi ** 5) / (kzero * rhoprime * 720) * (8 * (psiprime ** 4) * (11 - 24 * (tprime ** 2)) - 12 * (psiprime ** 3) * (21 - 71 * (tprime ** 2)) + 15 * (psiprime ** 2) * (15 - 98 * (tprime ** 2) + 15 * (tprime ** 4)) + 180 * psiprime * (5 * (tprime ** 2) - 3 * (tprime ** 4)) + 360 * (tprime ** 4));
    term_4 = tprime * Eprime * (chi ** 7) / (kzero * rhoprime * 40320) * (1385 + 3633 * (tprime ** 2) + 4095 * (tprime ** 4) + 1575 * (tprime ** 6));
    term1 = chi * (1 / math.cos(phiprime));
    term2 = (chi ** 3) * (1 / math.cos(phiprime)) / 6 * (psiprime + 2 * (tprime ** 2));
    term3 = (chi ** 5) * (1 / math.cos(phiprime)) / 120 * (-4 * (psiprime ** 3) * (1 - 6 * (tprime ** 2)) + (psiprime ** 2) * (9 - 68 * (tprime ** 2)) + 72 * psiprime * (tprime ** 2) + 24 * (tprime ** 4));
    term4 = (chi ** 7) * (1 / math.cos(phiprime)) / 5040 * (61 + 662 * (tprime ** 2) + 1320 * (tprime ** 4) + 720 * (tprime ** 6));

    latitude = (phiprime - term_1 + term_2 - term_3 + term_4) * 180 / math.pi;
    longitude = lambdazero + 180 / math.pi * (term1 - term2 + term3 - term4);  

#         outfile.write("{}, {}, {}\n".format(site_id, latitude, longitude))
    return latitude, longitude

In [3]:
df_orig = pd.read_csv(r'data/Average_Daily_Traffic_Counts_20190728.csv')
display_all(df_orig.head())
df = df_orig.copy()

,X,Y,OBJECTID,carr_way_no,road_id,road_name,start_name,end_name,location,latest,count_date,peak_hour,count_duration,adt,peaktraffic,pccar,pclcv,pcmcv,pchcvi,pchcvii,pcbus,pcheavy,NZTMX,NZTMY
0,1.753117e+06,5.915068e+06,4001,14721.0,31125,RICHARDSON RD (OWAIRAKA),HENDON AVE,O'DONNELL AVE,1218.0,No,1986-06-25T00:00:00.000Z,16:15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.753117e+06,5.915068e+06
1,1.753117e+06,5.915068e+06,4002,14721.0,31125,RICHARDSON RD (OWAIRAKA),HENDON AVE,O'DONNELL AVE,1218.0,No,1984-06-20T00:00:00.000Z,16:15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.753117e+06,5.915068e+06
2,1.753117e+06,5.915068e+06,4003,14721.0,31125,RICHARDSON RD (OWAIRAKA),HENDON AVE,O'DONNELL AVE,1218.0,No,1984-06-15T00:00:00.000Z,16:15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.753117e+06,5.915068e+06
3,1.753117e+06,5.915068e+06,4004,14721.0,31125,RICHARDSON RD (OWAIRAKA),HENDON AVE,O'DONNELL AVE,1218.0,No,1993-10-27T00:00:00.000Z,17:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.753117e+06,5.915068e+06
4,1.753117e+06,5.915068e+06,4005,14721.0,31125,RICHARDSON RD (OWAIRAKA),HENDON AVE,O'DONNELL AVE,1218.0,No,1993-10-27T00:00:00.000Z,17:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.753117e+06,5.915068e+06


In [4]:
temp = df_orig.apply(lambda row: nztm_to_lat_long(row['NZTMX'], row['NZTMY']), axis = 1)

In [5]:
temp2 = temp.apply(pd.Series)
temp2.columns = ['latitude', 'longitude']
df['latitude'] = temp2['latitude']
df['longitude'] = temp2['longitude']

In [7]:
df['count_date'] = df['count_date'].str[:10]
df['count_date'] = pd.to_datetime(df['count_date'])

In [8]:
df.isna().sum()

X                     0
Y                     0
OBJECTID              0
carr_way_no         252
road_id               0
road_name             0
start_name          515
end_name            586
location             81
latest                0
count_date            0
peak_hour         26452
count_duration    24116
adt                8151
peaktraffic       32121
pccar             23699
pclcv             23699
pcmcv             23703
pchcvi            23699
pchcvii           24633
pcbus             24237
pcheavy           22462
NZTMX                 0
NZTMY                 0
latitude              0
longitude             0
dtype: int64

In [9]:
# only consider 7-day traffic count measure
df2 = df[df.count_duration == '7D']
# consider only adt traffic measure (other measures have too many missing values)
# consider only longitude/latitude coordinate (drop X, Y, NZTMX, NZTMY)
cols_wanted = ['carr_way_no', 'road_id', 'road_name',
       'start_name', 'end_name', 'location', 'latest', 'count_date',
       'adt', 'latitude', 'longitude']
df2 = df2[cols_wanted]

In [10]:
df2.isna().sum()
# drop rows with missing values for now since we seem to have enough data
df2.dropna(inplace = True)
df2.isna().sum()

carr_way_no     148
road_id           0
road_name         0
start_name      409
end_name        440
location         81
latest            0
count_date        0
adt            1825
latitude          0
longitude         0
dtype: int64

carr_way_no    0
road_id        0
road_name      0
start_name     0
end_name       0
location       0
latest         0
count_date     0
adt            0
latitude       0
longitude      0
dtype: int64

In [11]:
df_road = pd.read_csv(r'data/Traffic_Management_Levels_20190728.csv')
df_road.head()

,OBJECTID,carr_way_no,road_id,road_name,start_name,end_name,cway_width,traff_manage_level,Shape__Length
0,2001,41976,41096,BABICH RD,WIDTH CHANGE (END K&C RHS),END,5.0,"< 10,000 vpd Rural/<15,000 vpd Urban",636.519615
1,2002,32589,41433,BABICH RD NORTH,METCALFE RD,PLATINUM RISE,10.0,<500 vpd (Low Volume),72.580121
2,2003,32605,41433,BABICH RD NORTH,PLATINUM RISE,SAUVIGNON AVE,10.0,<500 vpd (Low Volume),122.037043
3,2004,32606,41433,BABICH RD NORTH,SAUVIGNON AVE,END OF STAGE 1,10.0,<500 vpd (Low Volume),76.091004
4,2005,34909,41433,BABICH RD NORTH,END OF STAGE 1,END OF STAGE 2,10.0,<500 vpd (Low Volume),163.637715


In [12]:
df2.columns
df_road.columns
[col for col in df2.columns if col in df_road.columns]
# OBJECTID is the index, 
# carr_way_no is the id for a part of the road e.g. a raod can have many carriage way
# road id 

Index(['carr_way_no', 'road_id', 'road_name', 'start_name', 'end_name',
       'location', 'latest', 'count_date', 'adt', 'latitude', 'longitude'],
      dtype='object')

Index(['OBJECTID', 'carr_way_no', 'road_id', 'road_name', 'start_name',
       'end_name', 'cway_width', 'traff_manage_level', 'Shape__Length'],
      dtype='object')

['carr_way_no', 'road_id', 'road_name', 'start_name', 'end_name']

In [13]:
df2.shape
# df2.isna().sum()
df_road.shape
df_road.isna().sum()

(25755, 11)

(37474, 9)

OBJECTID                0
carr_way_no             0
road_id                 0
road_name               0
start_name            264
end_name              297
cway_width              0
traff_manage_level    179
Shape__Length           0
dtype: int64

In [14]:
cols = [col for col in df2.columns if col in df_road.columns]
df3 = df2.merge(df_road, how = 'left', on = cols)
df3.isna().sum()

carr_way_no            0
road_id                0
road_name              0
start_name             0
end_name               0
location               0
latest                 0
count_date             0
adt                    0
latitude               0
longitude              0
OBJECTID              14
cway_width            14
traff_manage_level    14
Shape__Length         14
dtype: int64

In [15]:
# a few roads don't have road info, drop these records
df3.dropna(inplace=True)

In [16]:
df3.drop(columns=['OBJECTID'], inplace = True)

In [17]:
df3.columns

Index(['carr_way_no', 'road_id', 'road_name', 'start_name', 'end_name',
       'location', 'latest', 'count_date', 'adt', 'latitude', 'longitude',
       'cway_width', 'traff_manage_level', 'Shape__Length'],
      dtype='object')

In [18]:
df3.shape

(25741, 14)

In [19]:
df3.to_csv(r'data\merged_data_20190728.csv', index = False)

In [20]:
df3.start_name.value_counts().head(10)

ISLAND NOSE MOTORWAY    634
END OF SEAL             319
URBAN/RURAL             173
GREAT NORTH RD          147
WHANGAPARAOA RD         115
TE ATATU RD              97
S HWAY 1                 95
SPEED DERESTRICTION      93
S HWAY 16                80
START OF SEAL            73
Name: start_name, dtype: int64

In [21]:
df.end_name.value_counts().head(10)

END OF SEAL            808
SPEED RESTRICTION      766
END                    427
CUL-DE-SAC             381
URBAN/RURAL            359
WIDTH CHANGE           311
START OF 2 LANES       303
GATE                   280
BEAUMONT ST            267
SPEED DERESTRICTION    258
Name: end_name, dtype: int64